##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Rede Neural Convolucional (CNN)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/cnn">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    Ver em TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Executar no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    Visualizar Código Fonte no GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Baixar notebook</a>
  </td>
</table>

Este tutorial demonstra o treinamento de uma simples [Rede Neural Convolucional] (https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) (CNN) para classificar [imagens CIFAR] (https://www.cs.toronto.edu/~kriz/cifar.html). Como este tutorial usa a [Keras Sequential API] (https://www.tensorflow.org/guide/keras/overview), a criação e o treinamento do nosso modelo levará apenas algumas linhas de código.

### Importando TensorFlow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### Baixar e preparar o conjunto de dados CIFAR10

O conjunto de dados CIFAR10 contém 60.000 imagens coloridas em 10 classes, com 6.000 imagens em cada classe. O conjunto de dados é dividido em 50.000 imagens de treinamento e 10.000 imagens de teste. As classes são mutuamente exclusivas e não há sobreposição entre elas.

In [0]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalizar os valores dos pixels entre 0 e 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Verificar os Dados

Para verificar se o conjunto de dados está correto, plote as 25 primeiras imagens do conjunto de treinamento e exiba o nome da classe abaixo de cada imagem.


In [0]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### Criar a base convolucional

As 6 linhas de código abaixo definem a base convolucional usando um padrão comum: uma pilha de [Conv2D] (https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) e [MaxPooling2D] (https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D).

Como entrada, uma CNN assume tensores de formato (altura da imagem, largura da imagem, canais de cores), ignorando o tamanho do lote. Se você é novo nessas dimensões, color_channels se refere a (R,G,B). Neste exemplo, você configurará nossa CNN para processar entradas de formato (32, 32, 3), que é o formato das imagens CIFAR. Você pode fazer isso passando o argumento `input_shape` para a nossa primeira camada.

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Vamos exibir a arquitetura do nosso modelo até agora.

In [0]:
model.summary()

Acima, você pode ver que a saída de todas as camadas Conv2D e MaxPooling2D é um tensor 3D de formato (altura, largura, canais). As dimensões de largura e altura tendem a encolher à medida que você avança na rede. O número de canais de saída para cada camada Conv2D é controlado pelo primeiro argumento (por exemplo, 32 ou 64). Normalmente, à medida que a largura e a altura diminuem, você pode permitir (computacionalmente) adicionar mais canais de saída em cada camada do Conv2D.

### Adicionar camadas Dense no topo

Para concluir nosso modelo, você alimentará o último tensor de saída da base convolucional (de formato (3,3,64)) em uma ou mais camadas densas para realizar a classificação. Camadas densas recebem vetores como entrada (que são 1D), enquanto a saída atual é um tensor 3D. Primeiro, você achatará (ou desenrolará) a saída 3D para 1D e depois adicionará uma ou mais camadas densas na parte superior. O CIFAR possui 10 classes de saída, portanto, você usa uma camada Dense final com 10 saídas e uma função de ativação softmax.

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Aqui está a arquitetura completa do nosso modelo.

In [0]:
model.summary()

Como você pode ver, nossas (3, 3, 64) saídas foram achatadas em vetores de forma (576) antes de passar por duas camadas dense.

### Compilar e treinar o modelo

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### Avaliar o modelo

In [0]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [0]:
print(test_acc)

Nossa CNN simples alcançou uma precisão de teste superior a 70%. Nada mal para algumas linhas de código! Para outro estilo da CNN, veja um exemplo usando a API de subclasse Keras e um `tf.GradientTape` [aqui] (https://www.tensorflow.org/tutorials/quickstart/advanced).